# SiLU / Swish — CNN & Object Detection için Uçtan Uca Notlar

Bu defter, **SiLU (Sigmoid Linear Unit)** aktivasyonunu (diğer adıyla **Swish**) temelden ileriye anlatır.

Özellikler:
- CNN ve detection bağlamı (YOLO tarzı ağlarda neden yaygın)
- Kod örnekleri **minimal**
- Matematik **hafif**: sadece gerekli kadar (türev/ileri-geri akışı için sezgi)

> Not: PyTorch’ta `nn.SiLU` kullanılır. “Swish” genelde aynı fonksiyona referans verir.

## 1) SiLU / Swish nedir?

SiLU fonksiyonu:

\[
\mathrm{SiLU}(x) = x \cdot \sigma(x)
\]

Burada \(\sigma(x)\) sigmoid fonksiyonudur:

\[
\sigma(x) = \frac{1}{1 + e^{-x}}
\]

### Neden önemli?
- ReLU gibi “keskin kapı” değil, **yumuşak (smooth)**
- Negatif tarafta tamamen sıfırlamaz (ReLU gibi “dead” yapma ihtimali daha az)
- Gradient akışı daha sürekli olabilir

In [1]:
import torch
import torch.nn as nn

act = nn.SiLU()  # Swish ile aynı kullanım (pratikte)
x = torch.tensor([-4.0, -2.0, -1.0, 0.0, 1.0, 2.0, 4.0])
print("x    :", x.tolist())
print("SiLU :", act(x).tolist())

x    : [-4.0, -2.0, -1.0, 0.0, 1.0, 2.0, 4.0]
SiLU : [-0.07194484025239944, -0.23840583860874176, -0.2689414322376251, 0.0, 0.7310585975646973, 1.7615940570831299, 3.9280550479888916]


## 2) ReLU / LeakyReLU / SiLU farkı (sezgisel)

- **ReLU:** negatifleri 0 yapar → hızlı ve basit ama bazı nöronlar “ölü” kalabilir.
- **LeakyReLU:** negatifte küçük eğim bırakır → gradient tamamen kesilmez.
- **SiLU:** negatifte “yumuşak” şekilde bastırır, pozitifte lineere yaklaşır → daha smooth optimizasyon.

SiLU’nun en önemli karakteri:
- **x negatifken** çıktı negatif ama küçük (tam sıfır değil)
- **x pozitifken** çıktı yaklaşık lineer (≈ x)

## 3) SiLU neden object detection’da çok kullanılıyor?

YOLO ailesi ve benzeri CNN tabanlı detector’larda SiLU’nun pratik avantajları:

- Aktivasyon **smooth** olduğu için optimizasyon bazen daha stabil olur
- Düşük seviyeli özelliklerde (kenar/texture) negatif değerlerin tamamen ölmemesi fayda verebilir
- BN/GN ile birlikte iyi çalışır
- Birçok modern backbone/neck tasarımında varsayılan aktivasyon haline geldi

**Ama:** Her zaman kazanmaz.
- Bazı görevlerde ReLU daha iyi olabilir
- SiLU hesap olarak ReLU’dan pahalıdır (sigmoid içerir)

## 4) Forward/backward sezgisi (çok hafif matematik)

\[
\mathrm{SiLU}(x) = x\sigma(x)
\]

Türev (bilmek faydalı):
\[
\frac{d}{dx}\mathrm{SiLU}(x) = \sigma(x) + x\sigma(x)(1-\sigma(x))
\]

### Buradan çıkan pratik sonuç
- Türev **her yerde** tanımlı ve süreklidir (ReLU gibi köşeli değil)
- Negatif bölgede türev 0’a tam çakılmaz → gradient akışı daha “nazik” kesilir

## 5) Minimal blok: Conv → BN → SiLU

CNN tarafında en klasik kullanım.

In [2]:
import torch
import torch.nn as nn

class ConvBNSiLU(nn.Module):
    def __init__(self, cin, cout, k=3, s=1, p=1):
        super().__init__()
        self.conv = nn.Conv2d(cin, cout, k, stride=s, padding=p, bias=False)
        self.bn = nn.BatchNorm2d(cout)
        self.act = nn.SiLU(inplace=True)

    def forward(self, x):
        return self.act(self.bn(self.conv(x)))

x = torch.randn(2, 16, 32, 32)
block = ConvBNSiLU(16, 32, s=2)
y = block(x)
y.shape

torch.Size([2, 32, 16, 16])

## 6) SiLU vs ReLU: ne zaman hangisi?

### SiLU tercih edilebilir:
- YOLO tarzı detector backbone/neck’lerinde baseline olarak
- Eğitim “hırçınsa” (loss dalgalanıyorsa)
- Küçük objelerde/ince detaylarda özellik taşımayı iyileştirmek istiyorsan

### ReLU tercih edilebilir:
- Çok hızlı inference gerekiyorsa
- Basit baseline veya mobil/edge optimizasyonu önemliyse
- Quantization / deployment hattın çok kısıtlıysa

### LeakyReLU arada bir çözümdür:
- ReLU kadar basit
- Negatifte gradient kesilmesini azaltır

## 7) Pratik ablation planı

SiLU eklerken:
1. Aynı modelin ReLU / LeakyReLU / SiLU versiyonlarını hazırla
2. Aynı seed, aynı dataloader, aynı epoch ile koştur
3. Kıyas metrikleri:
   - Test acc / mAP
   - Test loss
   - Eğitim stabilitesi (loss curve dalgalanması)
   - Hız (iter/s veya inference latency)

**Amaç:** “SiLU daha iyi mi?” değil, “benim senaryomda ne yapıyor?”u ölçmek.

## 8) Kısa özet

- SiLU/Swish = \(x \cdot \sigma(x)\)
- Smooth bir aktivasyondur; gradient akışı daha süreklidir
- Detection’da pratikte sık kullanılır
- ReLU’dan daha pahalıdır ama bazen performans/stabilite getirir